In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1122389,2021-03-26,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1122390,2021-03-27,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1122391,2021-03-28,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1122392,2021-03-29,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
30207,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
30209,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
30211,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
30213,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
30215,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
692910,2021-03-26,Arkansas,Arkansas,5001,2071,36.00,5000,Arkansas,AR,Arkansas,State,3017804
692912,2021-03-27,Arkansas,Arkansas,5001,2072,36.00,5000,Arkansas,AR,Arkansas,State,3017804
692914,2021-03-28,Arkansas,Arkansas,5001,2072,36.00,5000,Arkansas,AR,Arkansas,State,3017804
692916,2021-03-29,Arkansas,Arkansas,5001,2072,36.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1122389,2021-03-26,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1122390,2021-03-27,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1122391,2021-03-28,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1122392,2021-03-29,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-03-30') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
434,2021-03-30,Snohomish,Washington,32600,558.00,53061,WA,County,822083,3965.54,67.88
866,2021-03-30,Cook,Illinois,496856,10200.00,17031,IL,County,5150233,9647.25,198.05
1297,2021-03-30,Orange,California,266057,4740.00,6059,CA,County,3175692,8377.92,149.26
1727,2021-03-30,Maricopa,Arizona,524077,9657.00,4013,AZ,County,4485414,11684.03,215.30
2157,2021-03-30,Los Angeles,California,1219612,23110.00,6037,CA,County,10039107,12148.61,230.20
...,...,...,...,...,...,...,...,...,...,...,...
1121841,2021-03-30,Wheeler,Oregon,25,1.00,41069,OR,County,1332,1876.88,75.08
1122010,2021-03-30,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1122148,2021-03-30,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00
1122282,2021-03-30,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
434,2021-03-30,Snohomish,Washington,32600,558.00,53061,WA,County,822083,3965.54,67.88,67.88,3965.54
866,2021-03-30,Cook,Illinois,496856,10200.00,17031,IL,County,5150233,9647.25,198.05,198.05,9647.25
1297,2021-03-30,Orange,California,266057,4740.00,6059,CA,County,3175692,8377.92,149.26,149.26,8377.92
1727,2021-03-30,Maricopa,Arizona,524077,9657.00,4013,AZ,County,4485414,11684.03,215.30,215.30,11684.03
2157,2021-03-30,Los Angeles,California,1219612,23110.00,6037,CA,County,10039107,12148.61,230.20,230.20,12148.61
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1121841,2021-03-30,Wheeler,Oregon,25,1.00,41069,OR,County,1332,1876.88,75.08,75.08,1876.88
1122010,2021-03-30,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1122148,2021-03-30,Esmeralda,Nevada,38,0.00,32009,NV,County,873,4352.81,0.00,0.00,4352.81
1122282,2021-03-30,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
724477,2021-03-30,Hale,Alabama,2159,73.00,1065,AL,County,14651,14736.20,498.26,498.26,14736.20,1
788447,2021-03-30,Clarke,Alabama,3456,58.00,1025,AL,County,23622,14630.43,245.53,245.53,14630.43,2
567081,2021-03-30,Lowndes,Alabama,1361,53.00,1085,AL,County,9726,13993.42,544.93,544.93,13993.42,3
422106,2021-03-30,Franklin,Alabama,4184,82.00,1059,AL,County,31362,13340.99,261.46,261.46,13340.99,4
502904,2021-03-30,Etowah,Alabama,13643,342.00,1055,AL,County,102268,13340.44,334.42,334.42,13340.44,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
691817,2021-03-30,Sublette,Wyoming,703,7.00,56035,WY,County,9831,7150.85,71.20,71.20,7150.85,19
691449,2021-03-30,Converse,Wyoming,982,17.00,56009,WY,County,13822,7104.62,122.99,122.99,7104.62,20
859234,2021-03-30,Lincoln,Wyoming,1349,12.00,56023,WY,County,19830,6802.82,60.51,60.51,6802.82,21
955980,2021-03-30,Niobrara,Wyoming,150,2.00,56027,WY,County,2356,6366.72,84.89,84.89,6366.72,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
567081,2021-03-30,Lowndes,Alabama,1361,53.00,1085,AL,County,9726,13993.42,544.93,544.93,13993.42,3,1
724477,2021-03-30,Hale,Alabama,2159,73.00,1065,AL,County,14651,14736.20,498.26,498.26,14736.20,1,2
241451,2021-03-30,Walker,Alabama,7053,274.00,1127,AL,County,63521,11103.41,431.35,431.35,11103.41,28,3
611478,2021-03-30,Greene,Alabama,904,34.00,1063,AL,County,8111,11145.36,419.18,419.18,11145.36,26,4
566341,2021-03-30,Crenshaw,Alabama,1502,57.00,1041,AL,County,13772,10906.19,413.88,413.88,10906.19,29,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
610001,2021-03-30,Hot Springs,Wyoming,363,3.00,56017,WY,County,4413,8225.70,67.98,67.98,8225.70,17,19
859234,2021-03-30,Lincoln,Wyoming,1349,12.00,56023,WY,County,19830,6802.82,60.51,60.51,6802.82,21,20
837876,2021-03-30,Uinta,Wyoming,2125,12.00,56041,WY,County,20226,10506.28,59.33,59.33,10506.28,4,21
240320,2021-03-30,Teton,Wyoming,3593,9.00,56039,WY,County,23464,15312.82,38.36,38.36,15312.82,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,67.88,3965.54,15,22
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,67.88,3965.54,15,22
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,67.88,3965.54,15,22
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,67.88,3965.54,15,22
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,67.88,3965.54,15,22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1119750,2021-03-26,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1119751,2021-03-27,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1119752,2021-03-28,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1119753,2021-03-29,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
721473,2020-03-30,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,498.26,14736.20,2,1,1.00
721474,2020-03-31,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,498.26,14736.20,2,1,0.00
721475,2020-04-01,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,498.26,14736.20,2,1,0.00
721476,2020-04-02,Hale,Alabama,1,0.00,1065,AL,County,14651,6.83,0.00,498.26,14736.20,2,1,0.00
721477,2020-04-03,Hale,Alabama,3,0.00,1065,AL,County,14651,20.48,0.00,498.26,14736.20,2,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
873558,2021-03-26,Crook,Wyoming,422,11.00,56011,WY,County,7584,5564.35,145.04,145.04,5590.72,7,23,0.00
873559,2021-03-27,Crook,Wyoming,422,11.00,56011,WY,County,7584,5564.35,145.04,145.04,5590.72,7,23,0.00
873560,2021-03-28,Crook,Wyoming,422,11.00,56011,WY,County,7584,5564.35,145.04,145.04,5590.72,7,23,0.00
873561,2021-03-29,Crook,Wyoming,424,11.00,56011,WY,County,7584,5590.72,145.04,145.04,5590.72,7,23,2.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
564809,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,13993.42,1,3,1.00,0.00
564810,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,13993.42,1,3,0.00,0.00
564811,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,13993.42,1,3,0.00,0.00
564812,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,13993.42,1,3,0.00,0.00
564813,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,544.93,13993.42,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
562954,2021-03-26,Albany,Wyoming,3961,11.00,56001,WY,County,38880,10187.76,28.29,28.29,10208.33,23,7,9.00,0.00
562955,2021-03-27,Albany,Wyoming,3961,11.00,56001,WY,County,38880,10187.76,28.29,28.29,10208.33,23,7,0.00,0.00
562956,2021-03-28,Albany,Wyoming,3961,11.00,56001,WY,County,38880,10187.76,28.29,28.29,10208.33,23,7,0.00,0.00
562957,2021-03-29,Albany,Wyoming,3968,11.00,56001,WY,County,38880,10205.76,28.29,28.29,10208.33,23,7,7.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-03-30') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
288167,2021-03-30,Imperial,California,27442,711.00,6025,CA,County,181215,15143.34,392.35,392.35,15143.34,1,2,16.00,0.00
2157,2021-03-30,Los Angeles,California,1219612,23110.00,6037,CA,County,10039107,12148.61,230.20,230.20,12148.61,2,5,375.00,9.00
143611,2021-03-30,San Bernardino,California,290803,4335.00,6071,CA,County,2180085,13339.07,198.85,198.85,13339.07,3,4,88.00,3.00
504340,2021-03-30,Inyo,California,1372,35.00,6027,CA,County,18039,7605.74,194.02,194.02,7605.74,4,24,0.00,0.00
64277,2021-03-30,Stanislaus,California,58883,997.00,6099,CA,County,550660,10693.17,181.06,181.06,10693.17,5,9,79.00,1.00
79674,2021-03-30,Tulare,California,49039,815.00,6107,CA,County,466195,10518.99,174.82,174.82,10518.99,6,10,34.00,0.00
29817,2021-03-30,Riverside,California,294192,4297.00,6065,CA,County,2470546,11907.97,173.93,173.93,11907.97,7,6,129.00,24.00
51155,2021-03-30,San Joaquin,California,69796,1278.00,6077,CA,County,762148,9157.80,167.68,167.68,9157.80,8,19,246.00,19.00
29039,2021-03-30,Fresno,California,99085,1606.00,6019,CA,County,999101,9917.42,160.74,160.74,9917.42,9,14,81.00,0.00
611420,2021-03-30,Kings,California,22617,241.00,6031,CA,County,152940,14788.15,157.58,157.58,14788.15,10,3,7.00,0.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1065076,2021-03-30,Lassen,California,5652,1.00,6035,CA,County,30573,18486.90,3.27,3.27,18486.90,56,1,-1.00,0.00
288167,2021-03-30,Imperial,California,27442,711.00,6025,CA,County,181215,15143.34,392.35,392.35,15143.34,1,2,16.00,0.00
611420,2021-03-30,Kings,California,22617,241.00,6031,CA,County,152940,14788.15,157.58,157.58,14788.15,10,3,7.00,0.00
143611,2021-03-30,San Bernardino,California,290803,4335.00,6071,CA,County,2180085,13339.07,198.85,198.85,13339.07,3,4,88.00,3.00
2157,2021-03-30,Los Angeles,California,1219612,23110.00,6037,CA,County,10039107,12148.61,230.20,230.20,12148.61,2,5,375.00,9.00
29817,2021-03-30,Riverside,California,294192,4297.00,6065,CA,County,2470546,11907.97,173.93,173.93,11907.97,7,6,129.00,24.00
180513,2021-03-30,Kern,California,106155,1247.00,6029,CA,County,900202,11792.35,138.52,138.52,11792.35,14,7,28.00,8.00
457113,2021-03-30,Merced,California,30558,435.00,6047,CA,County,277680,11004.75,156.66,156.66,11004.75,11,8,37.00,0.00
64277,2021-03-30,Stanislaus,California,58883,997.00,6099,CA,County,550660,10693.17,181.06,181.06,10693.17,5,9,79.00,1.00
79674,2021-03-30,Tulare,California,49039,815.00,6107,CA,County,466195,10518.99,174.82,174.82,10518.99,6,10,34.00,0.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
288167,2021-03-30,Imperial,California,27442,711.00,6025,CA,County,181215,15143.34,392.35,392.35,15143.34,1,2,16.00,0.00
2157,2021-03-30,Los Angeles,California,1219612,23110.00,6037,CA,County,10039107,12148.61,230.20,230.20,12148.61,2,5,375.00,9.00
143611,2021-03-30,San Bernardino,California,290803,4335.00,6071,CA,County,2180085,13339.07,198.85,198.85,13339.07,3,4,88.00,3.00
504340,2021-03-30,Inyo,California,1372,35.00,6027,CA,County,18039,7605.74,194.02,194.02,7605.74,4,24,0.00,0.00
64277,2021-03-30,Stanislaus,California,58883,997.00,6099,CA,County,550660,10693.17,181.06,181.06,10693.17,5,9,79.00,1.00
79674,2021-03-30,Tulare,California,49039,815.00,6107,CA,County,466195,10518.99,174.82,174.82,10518.99,6,10,34.00,0.00
29817,2021-03-30,Riverside,California,294192,4297.00,6065,CA,County,2470546,11907.97,173.93,173.93,11907.97,7,6,129.00,24.00
51155,2021-03-30,San Joaquin,California,69796,1278.00,6077,CA,County,762148,9157.80,167.68,167.68,9157.80,8,19,246.00,19.00
29039,2021-03-30,Fresno,California,99085,1606.00,6019,CA,County,999101,9917.42,160.74,160.74,9917.42,9,14,81.00,0.00
611420,2021-03-30,Kings,California,22617,241.00,6031,CA,County,152940,14788.15,157.58,157.58,14788.15,10,3,7.00,0.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,392.35,15143.34,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,392.35,15143.34,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,392.35,15143.34,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,392.35,15143.34,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,392.35,15143.34,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3772,03/26/21,Lassen,5649,1.00,18477.09,3.27,3.27,18486.90,56,1,12.00,0.00
3773,03/27/21,Lassen,5649,1.00,18477.09,3.27,3.27,18486.90,56,1,0.00,0.00
3774,03/28/21,Lassen,5649,1.00,18477.09,3.27,3.27,18486.90,56,1,0.00,0.00
3775,03/29/21,Lassen,5653,1.00,18490.17,3.27,3.27,18486.90,56,1,4.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,392.35,15143.34,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,392.35,15143.34,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,392.35,15143.34,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,392.35,15143.34,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,392.35,15143.34,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3855,03/26/21,Kings,22578,241.00,14762.65,157.58,157.58,14788.15,10,3,17.00,2.00
3856,03/27/21,Kings,22588,241.00,14769.19,157.58,157.58,14788.15,10,3,10.00,0.00
3857,03/28/21,Kings,22600,241.00,14777.04,157.58,157.58,14788.15,10,3,12.00,0.00
3858,03/29/21,Kings,22610,241.00,14783.58,157.58,157.58,14788.15,10,3,10.00,0.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)